# Feature engineering

In [40]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
def z_score_group(x):
    return (x - x.mean()) / x.std()

bins = [-1, 2, 5, 12, 20]
labels = ['Bebê (0-2)', 'Pre-Escolar (2-5)', 'Escolar (5-12)', 'Adolescente (12+)']
df['Faixa_Etaria'] = pd.cut(df['IDADE'], bins=bins, labels=labels)

df['IMC_Z_Score_Proxy'] = df.groupby('Faixa_Etaria')['IMC'].transform(z_score_group)
df['FC_Z_Score_Proxy'] = df.groupby('Faixa_Etaria')['FC'].transform(z_score_group)

In [ ]:
df = pd.read_csv(r'C:\Users\João Pedro\Documents\UFG\MD\estudo_caso\data\UCMF_cleaned.csv')


df['HDA_lower'] = df['HDA'].str.lower()

df['Sintoma_Dor'] = df['HDA_lower'].str.contains('dor').astype(int)
df['Sintoma_Cianose'] = df['HDA_lower'].str.contains('cianose').astype(int)
df['Sintoma_Dispn'] = df['HDA_lower'].str.contains('dispneia|dispnéia').astype(int)
df['Checkup_Assintomatico'] = df['HDA_lower'].str.contains('assintomático|check-up').astype(int)

termos_hipertensao = ['Hipertensão', 'Elevada', 'HAS']
df['PPA_Alterada'] = df['PPA'].apply(lambda x: 1 if any(t in str(x) for t in termos_hipertensao) else 0)

df.head()

,Peso,Altura,IMC,IDADE,PULSOS,PPA,NORMAL X ANORMAL,B2,SOPRO,FC,MOTIVO,HDA,HDA_lower,Sintoma_Dor,Sintoma_Cianose,Sintoma_Dispn,Checkup_Assintomatico,PPA_Alterada
0,5.0,51.0,19.0,0.115068,Normais,Não Calculado,Anormal,Normal,Sistólico,112.0,palpitação/taquicardia/arritmia,palpitacao,palpitacao,0,0,0,0,0
1,3.5,50.0,14.0,0.016438,Normais,Não Calculado,Anormal,Normal,ausente,128.0,dispnéia,dispneia,dispneia,0,0,1,0,0
2,16.0,99.0,17.0,3.823288,Normais,Não Calculado,Anormal,Normal,Sistólico,88.0,check-up,assintomático,assintomático,0,0,0,1,0
3,8.1,65.0,19.0,0.484932,Normais,Não Calculado,Anormal,Normal,ausente,92.0,parecer cardiológico,assintomático,assintomático,0,0,0,1,0
4,40.0,151.0,18.0,12.427397,Normais,Não Calculado,Anormal,Normal,ausente,96.0,parecer cardiológico,dor precordial,dor precordial,1,0,0,0,0


In [ ]:
df['MOTIVO_lower'] = df['MOTIVO'].str.lower()

df['Motivo_Cirurgia'] = df['MOTIVO_lower'].str.contains('cirurgia').astype(int)
df['Motivo_Sopro'] = df['MOTIVO_lower'].str.contains('sopro').astype(int)
df['Motivo_Congenita'] = df['MOTIVO_lower'].str.contains('congenita|congênita').astype(int)


In [27]:
df.head()

,Peso,Altura,IMC,IDADE,PULSOS,PPA,NORMAL X ANORMAL,B2,SOPRO,FC,...,HDA_lower,Sintoma_Dor,Sintoma_Cianose,Sintoma_Dispn,Checkup_Assintomatico,PPA_Alterada,MOTIVO_lower,Motivo_Cirurgia,Motivo_Sopro,Motivo_Congenita
0,5.0,51.0,19.0,0.115068,Normais,Não Calculado,Anormal,Normal,Sistólico,112.0,...,palpitacao,0,0,0,0,0,palpitação/taquicardia/arritmia,0,0,0
1,3.5,50.0,14.0,0.016438,Normais,Não Calculado,Anormal,Normal,ausente,128.0,...,dispneia,0,0,1,0,0,dispnéia,0,0,0
2,16.0,99.0,17.0,3.823288,Normais,Não Calculado,Anormal,Normal,Sistólico,88.0,...,assintomático,0,0,0,1,0,check-up,0,0,0
3,8.1,65.0,19.0,0.484932,Normais,Não Calculado,Anormal,Normal,ausente,92.0,...,assintomático,0,0,0,1,0,parecer cardiológico,0,0,0
4,40.0,151.0,18.0,12.427397,Normais,Não Calculado,Anormal,Normal,ausente,96.0,...,dor precordial,1,0,0,0,0,parecer cardiológico,0,0,0


In [28]:
df.columns

Index(['Peso', 'Altura', 'IMC', 'IDADE', 'PULSOS', 'PPA', 'NORMAL X ANORMAL',
       'B2', 'SOPRO', 'FC', 'MOTIVO', 'HDA', 'HDA_lower', 'Sintoma_Dor',
       'Sintoma_Cianose', 'Sintoma_Dispn', 'Checkup_Assintomatico',
       'PPA_Alterada', 'MOTIVO_lower', 'Motivo_Cirurgia', 'Motivo_Sopro',
       'Motivo_Congenita'],
      dtype='object')

In [ ]:
correcoes = {
    'anormal': 'Anormal',
    'Normais': 'Normal',
    'Anormal': 'Anormal',
    'Normal': 'Normal'
}
df['NORMAL X ANORMAL'] = df['NORMAL X ANORMAL'].map(correcoes)

df['Target'] = df['NORMAL X ANORMAL'].map({'Anormal': 1, 'Normal': 0})

print(df['Target'].value_counts())

df = df.drop(columns=['NORMAL X ANORMAL'])

Target
0    6726
1    4952
Name: count, dtype: int64


In [ ]:
mapa_sopro = {
    'sistólico': 'Sistolico',  
    'Sistólico': 'Sistolico',  
    'ausente': 'Ausente',      
    'Não Informado': 'Ausente',
    'contínuo': 'Continuo',
    'Contínuo': 'Continuo',
    'diastólico': 'Diastolico',
    'Sistolico e diastólico': 'Sistolico_Diastolico' 
}

df['SOPRO'] = df['SOPRO'].map(mapa_sopro).fillna('Ausente')

print("Valores únicos após limpeza:")
print(df['SOPRO'].value_counts())

Valores únicos após limpeza:
SOPRO
Ausente                 7257
Sistolico               4362
Continuo                  47
Diastolico                 9
Sistolico_Diastolico       3
Name: count, dtype: int64


In [31]:
df['SOPRO'] = df['SOPRO'].replace('Não Informado', 'ausente')

df = pd.get_dummies(df, columns=['SOPRO'], prefix='SOPRO')

cols_sopro = [c for c in df.columns if 'SOPRO_' in c]
df[cols_sopro] = df[cols_sopro].astype(int)

In [ ]:
mapa_b2 = {
    'Normal': 'Normal',
    'Hiperfonética': 'Hiperfonetica', 
    'Desdob fixo': 'Desdobramento_Fixo', 
    'Única': 'Unica', 
    'Outro': 'Outro'
}
df['B2'] = df['B2'].map(mapa_b2).fillna('Outro')

df = pd.get_dummies(df, columns=['B2'], prefix='B2')

cols_b2 = [c for c in df.columns if 'B2_' in c]
df[cols_b2] = df[cols_b2].astype(int)

In [33]:
df.columns

Index(['Peso', 'Altura', 'IMC', 'IDADE', 'PULSOS', 'PPA', 'FC', 'MOTIVO',
       'HDA', 'HDA_lower', 'Sintoma_Dor', 'Sintoma_Cianose', 'Sintoma_Dispn',
       'Checkup_Assintomatico', 'PPA_Alterada', 'MOTIVO_lower',
       'Motivo_Cirurgia', 'Motivo_Sopro', 'Motivo_Congenita', 'Target',
       'SOPRO_Ausente', 'SOPRO_Continuo', 'SOPRO_Diastolico',
       'SOPRO_Sistolico', 'SOPRO_Sistolico_Diastolico',
       'B2_Desdobramento_Fixo', 'B2_Hiperfonetica', 'B2_Normal', 'B2_Outro',
       'B2_Unica'],
      dtype='object')

In [35]:
cols_to_drop_now = [
    'Peso', 'Altura', 'PULSOS', 
    'HDA', 'HDA_lower', 
    'MOTIVO', 'MOTIVO_lower', 
    'PPA'
]

df_cleaned = df.drop(columns=cols_to_drop_now)
df_cleaned.columns

Index(['IMC', 'IDADE', 'FC', 'Sintoma_Dor', 'Sintoma_Cianose', 'Sintoma_Dispn',
       'Checkup_Assintomatico', 'PPA_Alterada', 'Motivo_Cirurgia',
       'Motivo_Sopro', 'Motivo_Congenita', 'Target', 'SOPRO_Ausente',
       'SOPRO_Continuo', 'SOPRO_Diastolico', 'SOPRO_Sistolico',
       'SOPRO_Sistolico_Diastolico', 'B2_Desdobramento_Fixo',
       'B2_Hiperfonetica', 'B2_Normal', 'B2_Outro', 'B2_Unica'],
      dtype='object')

In [44]:
df_cleaned.to_csv(r'C:\Users\João Pedro\Documents\UFG\MD\estudo_caso\data\UCMF_feat_eng_sem_z_score.csv', index=False)

In [36]:
X = df_cleaned.drop(columns=['Target'])
y = df_cleaned['Target']

In [45]:
print("Colunas que ficaram no X (para divisão):")
print(X.columns.tolist())

Colunas que ficaram no X (para divisão):
['IMC', 'IDADE', 'FC', 'Sintoma_Dor', 'Sintoma_Cianose', 'Sintoma_Dispn', 'Checkup_Assintomatico', 'PPA_Alterada', 'Motivo_Cirurgia', 'Motivo_Sopro', 'Motivo_Congenita', 'SOPRO_Ausente', 'SOPRO_Continuo', 'SOPRO_Diastolico', 'SOPRO_Sistolico', 'SOPRO_Sistolico_Diastolico', 'B2_Desdobramento_Fixo', 'B2_Hiperfonetica', 'B2_Normal', 'B2_Outro', 'B2_Unica']


In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [46]:
def atribuir_faixa_etaria(df_input):
    bins = [-1, 2, 5, 12, 20, 100]
    labels = ['0-2', '2-5', '5-12', '12-20', '20+']
    return pd.cut(df_input['IDADE'], bins=bins, labels=labels)

X_train['Faixa_Etaria'] = atribuir_faixa_etaria(X_train)
X_test['Faixa_Etaria'] = atribuir_faixa_etaria(X_test)

referencia_imc = X_train.groupby('Faixa_Etaria')['IMC'].agg(['mean', 'std'])
referencia_fc = X_train.groupby('Faixa_Etaria')['FC'].agg(['mean', 'std'])

print("Tabela de Referência aprendida (Treino):")
print(referencia_imc)

Tabela de Referência aprendida (Treino):
                   mean        std
Faixa_Etaria                      
0-2           16.435489   2.936391
2-5           16.776895   4.170617
5-12          17.747549  12.989164
12-20         20.075255  26.176300
20+           23.121951   3.756296


C:\Users\João Pedro\AppData\Local\Temp\ipykernel_10156\692263683.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  referencia_imc = X_train.groupby('Faixa_Etaria')['IMC'].agg(['mean', 'std'])
C:\Users\João Pedro\AppData\Local\Temp\ipykernel_10156\692263683.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  referencia_fc = X_train.groupby('Faixa_Etaria')['FC'].agg(['mean', 'std'])


In [ ]:
def calcular_z_com_referencia(row, tabela_ref, col_valor):
    faixa = row['Faixa_Etaria']
    
    if pd.isna(faixa):
        return 0
        
    if faixa not in tabela_ref.index:
        return 0
        
    media = tabela_ref.loc[faixa, 'mean']
    desvio = tabela_ref.loc[faixa, 'std']
    
    if desvio == 0: return 0
    return (row[col_valor] - media) / desvio

X_train['IMC_Z'] = X_train.apply(lambda x: calcular_z_com_referencia(x, referencia_imc, 'IMC'), axis=1)
X_train['FC_Z'] = X_train.apply(lambda x: calcular_z_com_referencia(x, referencia_fc, 'FC'), axis=1)

X_test['IMC_Z'] = X_test.apply(lambda x: calcular_z_com_referencia(x, referencia_imc, 'IMC'), axis=1)
X_test['FC_Z'] = X_test.apply(lambda x: calcular_z_com_referencia(x, referencia_fc, 'FC'), axis=1)

cols_auxiliares = ['IMC', 'FC', 'IDADE', 'Faixa_Etaria'] 
X_train = X_train.drop(columns=['IMC', 'FC', 'Faixa_Etaria'])
X_test = X_test.drop(columns=['IMC', 'FC', 'Faixa_Etaria'])

X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

In [49]:
X_train.to_csv(r'C:\Users\João Pedro\Documents\UFG\MD\estudo_caso\data\X_train.csv', index=False)
X_test.to_csv(r'C:\Users\João Pedro\Documents\UFG\MD\estudo_caso\data\X_test.csv', index=False)
y_train.to_csv(r'C:\Users\João Pedro\Documents\UFG\MD\estudo_caso\data\y_train.csv', index=False)
y_test.to_csv(r'C:\Users\João Pedro\Documents\UFG\MD\estudo_caso\data\y_test.csv', index=False)